In [17]:
!pip install transformers
!pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import re
import nltk
from nltk import wordnet 
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [19]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
data = pd.read_csv('/content/drive/MyDrive/airbnb_labelled.csv')
data.head()

listing_id            id      date  reviewer_id reviewer_name  \
0    11391287  2.081364e+08  31/10/17     68485599           Ali   
1    50319142  5.744510e+17    3/3/22     81633638    Christophe   
2      272282  5.281465e+07   2/11/15      8528459       Rebecca   
3     4683548  4.758410e+17  18/10/21    190705793           Ron   
4     7838608  1.923088e+08   10/9/17     12034187      Giuseppe   

                                              Review     Category Sentiment  
0  Fantastic location and fantastic hosts for a v...     location  positive  
1  Super happy with this stylish and comfy apartm...  communicate  positive  
2  Mar made us feel very welcomed with great dire...  communicate  positive  
3  Andres is an amazing host! <br/>Not only the a...     accurate  positive  
4  A lovely room which was spacious and clean. Al...  cleanliness  positive

In [23]:
# positive = [4, 5]
# negative = [1, 2]
# neutral = [3]

# def sentiment(rating):
#   if rating in positive:
#     return "positive"
#   elif rating in negative:
#     return "negative"
#   else:
#     return "neutral"  

# data['Sentiment'] = data['Rating'].apply(sentiment)
# data.head()

In [8]:
def preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    '''
    Preprocess a string.
    :parameter
        :param text: string - name of column containing text
        :param lst_stopwords: list - list of stopwords to remove
        :param flg_stemm: bool - whether stemming is to be applied
        :param flg_lemm: bool - whether lemmitisation is to be applied
    :return
        cleaned text
    '''

    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    print(text)
    text = re.sub(r'[^A-z\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    print(lst_text)
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [24]:
lst_stopwords = nltk.corpus.stopwords.words("english")
print(lst_stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [25]:
data["review_clean"] = data["Review"].apply(lambda x: preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))
data.head()

Fantastic location and fantastic hosts for a very reasonable price, would definitely use again!
['fantastic', 'location', 'and', 'fantastic', 'hosts', 'for', 'a', 'very', 'reasonable', 'price', 'would', 'definitely', 'use', 'again']
Super happy with this stylish and comfy apartment for our family of 5. Easy check-in, hospitable service, great amenities and situated in a beautiful building. Loved our stay and we'll be back!
['super', 'happy', 'with', 'this', 'stylish', 'and', 'comfy', 'apartment', 'for', 'our', 'family', 'of', 'easy', 'checkin', 'hospitable', 'service', 'great', 'amenities', 'and', 'situated', 'in', 'a', 'beautiful', 'building', 'loved', 'our', 'stay', 'and', 'well', 'be', 'back']
Mar made us feel very welcomed with great directions, guestbook and assistance before our arrival.  We loved the residential feel to the building and the neighbourhood, with a train located just around the corner but we walked every where and great to get away from the crowds. 
<br/>
<br/>The 

listing_id            id      date  reviewer_id reviewer_name  \
0    11391287  2.081364e+08  31/10/17     68485599           Ali   
1    50319142  5.744510e+17    3/3/22     81633638    Christophe   
2      272282  5.281465e+07   2/11/15      8528459       Rebecca   
3     4683548  4.758410e+17  18/10/21    190705793           Ron   
4     7838608  1.923088e+08   10/9/17     12034187      Giuseppe   

                                              Review     Category Sentiment  \
0  Fantastic location and fantastic hosts for a v...     location  positive   
1  Super happy with this stylish and comfy apartm...  communicate  positive   
2  Mar made us feel very welcomed with great dire...  communicate  positive   
3  Andres is an amazing host! <br/>Not only the a...     accurate  positive   
4  A lovely room which was spacious and clean. Al...  cleanliness  positive   

                                        review_clean  
0  fantastic location fantastic host reasonable p...  
1  super happy stylish comfy apartment family eas...  
2  mar made u feel welcomed great direction guest...  
3  andres amazing host brnot apartment see pictur...  
4  lovely room spacious clean also great support ...

In [ ]:
# training_text = '\n'.join(data['review_clean'].to_list())
# training_text

In [26]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import torch
import numpy as np
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
sentiment_labels = {'positive':2,
          'negative':1,
          'neutral':0
          }

class Sentiment_Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels = [sentiment_labels[label] for label in df['Sentiment']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['review_clean']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [57]:
category_labels = {'accurate':0,
          'cleanliness':1,
          'communicate':2,
          'value':3,
          'location':4
          }

class Category_Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels = [category_labels[label] for label in df['Category']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['review_clean']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [58]:
np.random.seed(112)
data_train, data_val, data_test = np.split(data.sample(frac=1, random_state=42), 
                                     [int(.6*len(data)), int(.8*len(data))])

In [59]:
print(len(data_train),len(data_val), len(data_test))

180 60 60


In [60]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [61]:
from torch.optim import Adam
from tqdm import tqdm

def train_category(model, train_data, val_data, learning_rate, epochs):

    train, val = Category_Dataset(train_data), Category_Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    
    device = torch.device("cuda" if use_cuda else "cpu")
    # device = torch.device("cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')

In [62]:
EPOCHS = 5
model_category = BertClassifier()
LR = 1e-6
              
train_category(model_category, data_train, data_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 90/90 [00:18<00:00,  4.74it/s]


Epochs: 1 | Train Loss:  0.816                 | Train Accuracy:  0.261                 | Val Loss:  0.841                 | Val Accuracy:  0.167


100%|██████████| 90/90 [00:18<00:00,  4.74it/s]


Epochs: 2 | Train Loss:  0.778                 | Train Accuracy:  0.289                 | Val Loss:  0.755                 | Val Accuracy:  0.417


100%|██████████| 90/90 [00:18<00:00,  4.75it/s]


Epochs: 3 | Train Loss:  0.766                 | Train Accuracy:  0.322                 | Val Loss:  0.785                 | Val Accuracy:  0.250


100%|██████████| 90/90 [00:19<00:00,  4.65it/s]


Epochs: 4 | Train Loss:  0.742                 | Train Accuracy:  0.389                 | Val Loss:  0.771                 | Val Accuracy:  0.350


100%|██████████| 90/90 [00:19<00:00,  4.50it/s]


Epochs: 5 | Train Loss:  0.736                 | Train Accuracy:  0.439                 | Val Loss:  0.768                 | Val Accuracy:  0.400


In [63]:
import pickle

f = open('/content/drive/MyDrive/bert_classifier_category.model', 'wb')
pickle.dump(model_category, f)
f.close()

In [64]:
def train_sentiment(model, train_data, val_data, learning_rate, epochs):

    train, val = Sentiment_Dataset(train_data), Sentiment_Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    
    device = torch.device("cuda" if use_cuda else "cpu")
    # device = torch.device("cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  

In [65]:
EPOCHS = 5
model_sentiment = BertClassifier()
LR = 1e-6
              
train_sentiment(model_sentiment, data_train, data_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 90/90 [00:19<00:00,  4.60it/s]


Epochs: 1 | Train Loss:  0.832                 | Train Accuracy:  0.150                 | Val Loss:  0.819                 | Val Accuracy:  0.183


100%|██████████| 90/90 [00:20<00:00,  4.49it/s]


Epochs: 2 | Train Loss:  0.760                 | Train Accuracy:  0.333                 | Val Loss:  0.649                 | Val Accuracy:  0.717


100%|██████████| 90/90 [00:19<00:00,  4.56it/s]


Epochs: 3 | Train Loss:  0.544                 | Train Accuracy:  0.794                 | Val Loss:  0.429                 | Val Accuracy:  0.917


100%|██████████| 90/90 [00:19<00:00,  4.65it/s]


Epochs: 4 | Train Loss:  0.372                 | Train Accuracy:  0.956                 | Val Loss:  0.301                 | Val Accuracy:  0.967


100%|██████████| 90/90 [00:19<00:00,  4.62it/s]


Epochs: 5 | Train Loss:  0.253                 | Train Accuracy:  0.956                 | Val Loss:  0.188                 | Val Accuracy:  0.967


In [66]:
f = open('/content/drive/MyDrive/bert_classifier_sentiment.model', 'wb')
pickle.dump(model_sentiment, f)
f.close()

In [15]:
import pickle

f = open('/content/drive/MyDrive/bert_classifier.model', 'rb')
model = pickle.load(f)

In [72]:
# test_category = Category_Dataset(data_test)

test_dataloader_category = torch.utils.data.DataLoader(test_category, batch_size=1)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
# device = torch.device("cpu")

if use_cuda:
  model_category = model_category.cuda()

category_results = []
category_labels = ['accurate', 'cleanliness', 'communicate', 'value', 'location']

with torch.no_grad():
  for test_input, test_label in test_dataloader_category:
    test_label = test_label.to(device)
    mask = test_input['attention_mask'].to(device)
    input_id = test_input['input_ids'].squeeze(1).to(device)

    output = model_category(input_id, mask)
    #print(output)
    category_results.append(category_labels[output.argmax(dim=1)])

print(category_results)

['location', 'location', 'location', 'location', 'value', 'location', 'location', 'location', 'location', 'location', 'communicate', 'location', 'location', 'location', 'location', 'value', 'location', 'value', 'location', 'communicate', 'location', 'location', 'value', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'value', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location', 'location']


In [73]:
with open('bert_classifier_y_pred_category.txt', 'w') as f:
  for item in category_results:
    f.write('%s\n' % item)

In [70]:
# test_sentiment = Sentiment_Dataset(data_test)

test_dataloader_sentiment = torch.utils.data.DataLoader(test_sentiment, batch_size=1)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
# device = torch.device("cpu")

if use_cuda:
  model_sentiment = model_sentiment.cuda()

sentiment_results = []
sentiment_labels = ['neutral', 'negative', 'positive']

with torch.no_grad():
  for test_input, test_label in test_dataloader_category:
    test_label = test_label.to(device)
    mask = test_input['attention_mask'].to(device)
    input_id = test_input['input_ids'].squeeze(1).to(device)

    output = model_sentiment(input_id, mask)
    #print(output)
    sentiment_results.append(sentiment_labels[output.argmax(dim=1)])

print(sentiment_results)

['positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive']


In [74]:
with open('bert_classifier_y_pred_sentiment.txt', 'w') as f:
  for item in category_results:
    f.write('%s\n' % item)